# Key word extraction algorithm for claims

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os, json
from helper_function import count_valid_posts, get_claims, get_premises
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

In [3]:
with open('../data/v4/Should-I-invest-in-Bitcoin_with_labels_v4_70_filled.json', 'r') as f:
    file_content = json.load(f)
print('Get valid posts: ', count_valid_posts(file_content))
claims = get_claims(file_content)
premises = get_premises(file_content)


Get valid posts:  (41, 70)


## Keyword extraction algorithms

In [4]:
# from keyword_extractor import TfIdfExtractor

# if_idf = TfIdfExtractor()

In [5]:
# if_idf.extract_keywords(premises)

## KeyBERT Extractor

In [6]:
from keyword_extractor import KeyBERTExtractor

In [7]:
keybert_extractor = KeyBERTExtractor()

In [8]:
# keybert_extractor.extract_keywords(premises)

## Extract Keywords under a certain center claim

In [9]:
# first build a list of supporting premises
claims_with_support = {}
for l in get_premises(file_content, return_list_of_strings=False):
    # l is a list of dict objects
    for o in l:
        clm, prm = o['supportClaim'], o['content']
        if not clm in claims_with_support:
            claims_with_support[clm] = []
        claims_with_support[clm].append(prm)

claim_center_relation = {}
for l in get_claims(file_content, return_list_of_strings=False):
    # l is a list of dict objects
    for o in l:
        clm, sc = o['content'], o['claimCenter']
        if not sc in claim_center_relation:
            claim_center_relation[sc] = []
        claim_center_relation[sc].append(clm)

claimCenter_premise = {}
for cs, c_list in claim_center_relation.items():
    if not cs in claimCenter_premise:
        claimCenter_premise[cs] = []
    for c in c_list:
        if c in claims_with_support:
            claimCenter_premise[cs].extend(claims_with_support[c])


In [10]:
output_data = []
for k,v in claimCenter_premise.items():
    term = {
        'content' : k,
        'keywords' : []
    }
    kws = keybert_extractor.extract_keywords(' '.join(v), keep_term=15)
    kws.sort(key=lambda a: a[1], reverse=True)
    term['keywords'] = kws
    output_data.append(term)
    # break

In [11]:
output_data

[{'content': 'It’s not too late to invest.',
  'keywords': [('bitcoin', 0.5662),
   ('investing', 0.4897),
   ('liquidity', 0.341),
   ('inflation', 0.3356),
   ('market', 0.3326),
   ('money', 0.2826),
   ('opportunity', 0.275),
   ('date', 0.1964),
   ('rethink', 0.1926),
   ('bought', 0.1832),
   ('open', 0.1817),
   ('whether', 0.1792),
   ('know', 0.1504),
   ('everything', 0.1037),
   ('vessel', 0.0647)]},
 {'content': 'Bitcoin is pretty useless. But so is gold.',
  'keywords': [('bitcoin', 0.5469),
   ('inflation', 0.3919),
   ('scarcity', 0.3134),
   ('scarce', 0.2717),
   ('gold', 0.2684),
   ('investment', 0.2657),
   ('hedge', 0.2561),
   ('argue', 0.2386),
   ('intrinsic', 0.2171),
   ('earth', 0.1934),
   ('useful', 0.1903),
   ('scam', 0.1798),
   ('horses', 0.1762),
   ('millenia', 0.1068),
   ('abruptly', 0.098)]},
 {'content': 'Invest in Bitcoin, only if you are okay to loss all.',
  'keywords': [('futures', 0.4698),
   ('bitcoin', 0.3795),
   ('invest', 0.3657),
   ('

In [12]:
# x = keybert_extractor.extract_keywords(' '.join(v), keep_term=15)
# x.sort(key=lambda a : a[1])
# x


In [13]:
with open('../data/newest_data(rolling update)/claim center and statistics/keywords.json', 'w') as f:
    json.dump(output_data, fp=f, indent=4)
